In [ ]:
!pip install tensorflow==1.14
!pip install keras==2.2.5
!pip install mrcnn
!pip install h5py==2.10.0

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 109.3 MB 62 kB/s 
     |████████████████████████████████| 488 kB 67.4 MB/s 
     |████████████████████████████████| 3.1 MB 61.4 MB/s 
     |████████████████████████████████| 50 kB 7.3 MB/s 
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.9.0
    Uninstalling tensorflow-estimator-2.9.0:
      Successfully uninstalled tensorflow-estimator-2.9.0
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.9.1
    Uninstalling tensorboard-2.9.1:
      Successfully uninstalled tensorboard-2.9.1
  Attempting uninstall: tensorflow
    Found existing installation: tensorflow 2.9.2
    Uninstalling tensorflow-2.9.2:
      Successfully uninstalled tensorflow-2.9.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour i

In [ ]:
import os
import sys
import json
import datetime
import numpy as np
import skimage.draw
import cv2
import random
import math
import re
import time
import tensorflow as tf
from shapely.geometry import Polygon
from skimage.measure import find_contours
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import matplotlib.image as mpimg

from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images
from mrcnn.visualize import display_instances
import mrcnn.model as modellib
from mrcnn.model import log
from mrcnn.config import Config
from mrcnn import model as modellib, utils

/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.7/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarning: Passing (type, 1) or 

In [ ]:
from google.colab import drive

In [ ]:
drive.mount('/content/drive/')
ROOT_DIR = '/content/drive/MyDrive/pythoncode/insuranceApp'

Mounted at /content/drive/


In [ ]:
DEFAULT_LOGS_DIR = os.path.join(ROOT_DIR, "logs")

MODEL_DIR = os.path.join(ROOT_DIR, "logs")

WEIGHTS_PATH = "/content/drive/MyDrive/pythoncode/insuranceApp/logs/object20221106T0743/mask_rcnn_object_0011.h5"

In [ ]:
class CustomConfig(Config):

    NAME = "object"
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1
    NUM_CLASSES = 1 + 2
    STEPS_PER_EPOCH = 50
    DETECTION_MIN_CONFIDENCE = 0.9

In [ ]:
class CustomDataset(utils.Dataset):
    def load_custom(self, dataset_dir, subset):

        self.add_class("object", 1, "scratch")
        self.add_class("object", 2, "dent")
        
        assert subset in ["train", "val"]
        dataset_dir = os.path.join(dataset_dir + subset)

        annotations1 = json.load(open(os.path.join(dataset_dir + '/' + "via_region_data_json.json"),'r',encoding="utf8",errors='ignore'))
        annotations = list(annotations1.values())

        annotations = [a for a in annotations if a['regions']]
        
        for a in annotations:
            
            polygons = [r['shape_attributes'] for r in a['regions']]
            objects = [s['region_attributes']['name'] for s in a['regions']]
            print("objects: ",objects)
            name_dict = {"scratch": 1, "dent": 2}

            num_ids = [name_dict[a] for a in objects]

            print("numids", num_ids)

            image_path = os.path.join(dataset_dir, a['filename'])
            image = skimage.io.imread(image_path)
            height, width = image.shape[:2]

            self.add_image(
                "object",
                image_id=a['filename'],
                path=image_path,
                width=width, height=height,
                polygons=polygons,
                num_ids=num_ids)
            
    def load_mask(self, image_id):
        
        image_info = self.image_info[image_id]
        if image_info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        
        info = self.image_info[image_id]
        if info["source"] != "object":
            return super(self.__class__, self).load_mask(image_id)
        num_ids = info['num_ids']
        mask = np.zeros([info["height"], info["width"], len(info["polygons"])],
                        dtype=np.uint8)
        for i, p in enumerate(info["polygons"]):
            rr, cc = skimage.draw.polygon(p['all_points_y'], p['all_points_x'])
            mask[rr, cc, i] = 1

        num_ids = np.array(num_ids, dtype=np.int32)
        return mask, num_ids

    def image_reference(self, image_id):

        info = self.image_info[image_id]
        if info["source"] == "object":
            return info["path"]
        else:
            super(self.__class__, self).image_reference(image_id)

In [ ]:
TEST_MODE = "inference"

def get_ax(rows=1, cols=1, size=16):    
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [ ]:
CUSTOM_DIR = "/content/drive/MyDrive/pythoncode/insuranceApp/dataset/"
dataset = CustomDataset()
dataset.load_custom(CUSTOM_DIR, "val")
dataset.prepare()
print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

objects:  ['scratch', 'scratch', 'scratch']
numids [1, 1, 1]
objects:  ['scratch']
numids [1]
objects:  ['scratch']
numids [1]
objects:  ['scratch']
numids [1]
objects:  ['scratch', 'scratch']
numids [1, 1]
objects:  ['scratch']
numids [1]
objects:  ['scratch']
numids [1]
objects:  ['scratch']
numids [1]
objects:  ['scratch']
numids [1]
objects:  ['scratch']
numids [1]
objects:  ['scratch']
numids [1]
objects:  ['scratch', 'scratch']
numids [1, 1]
objects:  ['scratch']
numids [1]
objects:  ['scratch']
numids [1]
objects:  ['scratch']
numids [1]
objects:  ['dent']
numids [2]
objects:  ['dent', 'scratch', 'dent']
numids [2, 1, 2]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent', 'scratch']
numids [2, 1]
objects:  ['dent']
numids [2]
objects:  ['scratch']
numids [1]
objects:  ['dent']
numids [2]
objects:  ['dent']
numids [2]
objects:  ['dent'

In [ ]:


config = CustomConfig()
#LOAD MODEL. Create model in inference mode
model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR, config=config)










Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
box_ind is deprecated, use box_indices instead


Instructions for updating:
Use `tf.cast` instead.


In [ ]:


# Load COCO weights Or, load the last model you trained
weights_path = WEIGHTS_PATH
# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)



Loading weights  /content/drive/MyDrive/pythoncode/insuranceApp/logs/object20221106T0743/mask_rcnn_object_0011.h5
Re-starting from epoch 11


In [ ]:
# print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

# This is for predicting images which are not present in dataset
path_to_new_image = '/content/drive/MyDrive/pythoncode/insuranceApp/dataset/test/test3.jpg'
image1 = mpimg.imread(path_to_new_image)

# Run object detection
#print(len([image1]))
results1 = model.detect([image1], verbose=0)


# Display results
ax = get_ax(1)
r1 = results1[0]

def save_co_ordinates(image, boxes, masks, class_ids, class_names):
    image = image.split("/")[-1]
    image_data = []

    for i in range(boxes.shape[0]):

        # class_id = class_ids[i]
        # label = class_names[class_id]

        mask = masks[:, :, i]
        padded_mask = np.zeros(
            (mask.shape[0] + 2, mask.shape[1] + 2), dtype=np.uint8)
        padded_mask[1:-1, 1:-1] = mask
        contours = find_contours(padded_mask, 0.5)
        for verts in contours:
            verts = np.fliplr(verts) - 1
            list_co_ordinates = np.moveaxis(verts, 1, 0).tolist()

            region = {"shape_attributes": {"all_points_x": list_co_ordinates[0],
                                           "all_points_y": list_co_ordinates[1]},
                      }
            image_data.append(region)
    data = {"filename": image, "regions": image_data, "count": boxes.shape[0]}
    return data

data = save_co_ordinates(path_to_new_image, r1['rois'], r1['masks'], r1['class_ids'], ['scratch','dent'])

for i in range(data['count']):
  x = data['regions'][i]['shape_attributes']['all_points_x']
  y = data['regions'][i]['shape_attributes']['all_points_y']
  pgon = Polygon(zip(x, y))
  print(pgon.area)

print(data['count'])

visualize.display_instances(image1, r1['rois'], r1['masks'], r1['class_ids'],
dataset.class_names, r1['scores'], ax=ax, title="Predictions1")

NameError: ignored

In [ ]:
# from pycocotools import mask

In [ ]:
# from skimage import measure

9599.5
10.5
2


In [ ]:
!pip install shapely

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
